In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, K2Score, HillClimbSearch
from pgmpy.readwrite import BIFWriter
import numpy as np
import dask.dataframe as dd
# Caricare il dataset
df3 = pd.read_csv('df_bayesiana3.csv')

print(df3)
df3 = df3.drop(columns = ['Unnamed: 0'])

df3.replace({7: -1, 9: -1, 77 : -1, 99 : -1, 777: -1, 999: -1, 7777: -1, 9999: -1 ,'nan': -1}, inplace=True)
df3.replace(-1.0, np.nan, inplace=True)
# Contare i NaN per colonna
nan_counts = df3.isna().sum()
nan_count_col1 = df3['DIABTYPE'].isna().sum()
print(f'Numero di NaN in col1: {nan_count_col1}')
# Determinare la soglia
soglia = 120000  # Modificare questo valore in base alla soglia desiderata

# Se il numero di valori mancanti in una colonna è maggiore della soglia, eliminare la colonna
df_cleaned = df3.drop(columns=nan_counts[nan_counts > soglia].index)
df_cleaned = df_cleaned.dropna(thresh=df_cleaned.shape[1] - 0)
df_sampled = df_cleaned.astype({col: 'float32' for col in df_cleaned.select_dtypes(include='float64').columns})

#df_sampled = df_cleaned.replace(np.nan, -1.0)
print(df_sampled)
df_sampled = df_sampled.drop(columns = ['SMOKE100','INCOME3','_AGEG5YR.1', 'EDUCA'])
print(df_sampled.dtypes)

#df3 = df3.drop(columns=['LANDSEX1','COVIDINT', 'PFPPRVN4','PFPPRVN4','MARJEAT', 'HPVADSHT','BLDSTFIT','CDHELP'])
#df3 = df3.fillna(-1)
# Dividere il dataset in train e test
train_data, test_data = train_test_split(df_sampled, test_size=0.2, random_state=42)
# Definire la struttura della rete bayesiana
# Utilizzare Hill-Climbing per apprendere la struttura
hc = HillClimbSearch(df_sampled)
best_model_structure = hc.estimate(max_iter = 100000, scoring_method=K2Score(df_sampled), max_indegree = 2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")
# Stampa la struttura appresa
#print("Archi della struttura appresa:", best_model.edges())

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("modello creato")

# Verifica che il modello sia stato creato correttamente
if model is None:
    raise ValueError("Errore nella creazione del modello di rete bayesiana.")

# Addestrare la rete bayesiana con i dati di training
try:
    model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello: {e}')
    raise

# Stampa delle CPD
#for cpd in model.get_cpds():
 #   print(cpd)
# Salvare il modello addestrato in formato .bif
#writer = BIFWriter(model)
#writer.write_bif(filename='trained_network.bif')


# Salvare i dati di test per utilizzarli successivamente
#test_data.to_csv('path_to_save_test_data.csv', index=False)

print("Modello addestrato e salvato con successo.")

In [ ]:
dfStat = pd.read_sas('LLCP2022.XPT')

In [ ]:
count_age = dfStat.groupby(['_AGEG5YR', 'DIABETE4']).size().reset_index(name='count')
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(count_age)

In [ ]:
count_sex = dfStat.groupby(['_SEX', 'DIABETE4']).size().reset_index(name='count')
print(count_sex)
# Creazione di un DataFrame pivot per il grafico a barre impilate
pivot_df = count_sex.pivot(index='_SEX', columns='DIABETE4', values='count').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set3')

# Aggiunta di etichette e titolo
plt.title('Distribuzione del Diabete per Sesso')
plt.xlabel('Sesso')
plt.ylabel('Conteggio')
plt.legend(title='Diabete')
plt.xticks(rotation=0)
plt.savefig('sessodiabete.jpeg', format="jpeg")
plt.show()
total_subjects = dfStat.shape[0]
print(f'Totale soggetti nel dataset: {total_subjects}')
total_count = count_sex['count'].sum()
print(f"Totale soggetti nel dataset: {total_count}")

In [ ]:
count_sex = dfStat.groupby(['_SEX', 'DIABTYPE']).size().reset_index(name='count')
print(count_sex)

count_sex = dfStat.groupby(['_SEX', 'DIABTYPE']).size().reset_index(name='count')
print(count_sex)
# Creazione di un DataFrame pivot per il grafico a barre impilate
pivot_df = count_sex.pivot(index='_SEX', columns='DIABTYPE', values='count').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set3')

# Aggiunta di etichette e titolo
plt.title('Distribuzione del tipo di Diabete per Sesso')
plt.xlabel('Sesso')
plt.ylabel('Conteggio')
plt.legend(title='Tipo di Diabete')
plt.xticks(rotation=0)
plt.savefig('sessotipodiabete.jpeg', format="jpeg")
plt.show()
total_subjects = dfStat.shape[0]
print(f'Totale soggetti nel dataset: {total_subjects}')
total_count = count_sex['count'].sum()
print(f"Totale soggetti nel dataset: {total_count}")


count_age = dfStat.groupby(['_AGEG5YR', 'DIABTYPE']).size().reset_index(name='count')
print(count_age)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
count_age = dfStat.groupby(['_AGEG5YR', 'DIABETE4']).size().reset_index(name='count')
print(count_age)

sns.set(style="whitegrid")

# Creazione del bar plot
plt.figure(figsize=(8, 6))
sns.barplot(x='_AGEG5YR', y='count', hue='DIABETE4', data=count_age, palette='Set2')

# Aggiunta di etichette e titolo
plt.title('Conteggio deL Diabete per età')
plt.xlabel('Età')
plt.ylabel('Conteggio')
plt.legend(title='Diabete')
plt.savefig('etadiabete.jpeg', format="jpeg")
plt.show()

In [ ]:
for sex in count_sex['_SEX'].unique():
    subset = count_sex[count_sex['_SEX'] == sex]
    
    # Creazione del pie chart
    plt.figure(figsize=(6, 6))
    plt.pie(subset['count'], labels=subset['DIABTYPE'], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('Set2'))
    plt.title(f'Distribuzione del Tipo di Diabete per {sex}')
    plt.axis('equal')
    plt.show()

In [ ]:
# Creazione di un DataFrame pivot per il grafico a barre impilate
pivot_df = count_sex.pivot(index='_SEX', columns='DIABTYPE', values='count').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set3')

# Aggiunta di etichette e titolo
plt.title('Distribuzione dei Tipi di Diabete per Sesso')
plt.xlabel('Sesso')
plt.ylabel('Conteggio')
plt.legend(title='Tipo di Diabete')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Creazione di un DataFrame pivot per il grafico a barre impilate
pivot_df = count_sex.pivot(index='_SEX', columns='DIABETE4', values='count').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set3')

# Aggiunta di etichette e titolo
plt.title('Distribuzione del Diabete per Sesso')
plt.xlabel('Sesso')
plt.ylabel('Conteggio')
plt.legend(title='Diabete')
plt.xticks(rotation=0)
plt.savefig('rete.jpeg', format="jpeg")
plt.show()

In [ ]:
# Creazione di un DataFrame pivot per il grafico a barre impilate
pivot_df = count_age.pivot(index='_AGEG5YR', columns='DIABETE4', values='count').fillna(0)

# Plot
pivot_df.plot(kind='bar', stacked=True, figsize=(8, 6), colormap='Set3')

# Aggiunta di etichette e titolo
plt.title('Distribuzione del Diabete per Età')
plt.xlabel('Età')
plt.ylabel('Conteggio')
plt.legend(title='Diabete')
plt.xticks(rotation=0)
plt.savefig('etasesso.jpeg', format="jpeg")
plt.show()